# GraphQL

>[GraphQL](https://graphql.org/) 是一种用于 API 的查询语言，也是在你的数据上执行这些查询的运行时。`GraphQL` 提供了对你 API 中数据的完整且易于理解的描述，使客户端能够请求他们所需的确切内容，不多也不少，从而更轻松地随着时间的推移演进 API，并支持强大的开发工具。

通过将 `BaseGraphQLTool` 包含在提供给 Agent 的工具列表中，你可以授予 Agent 的能力，使其能够根据你的任何需求从 GraphQL API 查询数据。

本 Jupyter Notebook 演示了如何将 `GraphQLAPIWrapper` 组件与 Agent 一起使用。

在本例中，我们将使用公开的 `Star Wars GraphQL API`，其端点为：https://swapi-graphql.netlify.app/graphql 。

首先，你需要安装 `httpx` 和 `gql` Python 包。

In [1]:
pip install httpx gql > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade --quiet  langchain-community

Note: you may need to restart the kernel to use updated packages.


现在，让我们创建一个具有指定星球大战 API 端点的 `BaseGraphQLTool` 实例，并用该工具初始化一个 `Agent`。

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [4]:
from langchain_community.agent_toolkits.load_tools import load_tools

tools = load_tools(
    ["graphql"],
    graphql_endpoint="https://swapi-graphql.netlify.app/graphql",
)

In [5]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent("openai:gpt-4.1-mini", tools)

现在，我们可以使用 Agent 对 Star Wars GraphQL API 运行查询。让我们让 Agent 列出所有的星球大战电影及其发行日期。

In [6]:
graphql_fields = """allFilms {
    films {
      title
      director
      releaseDate
      speciesConnection {
        species {
          name
          classification
          homeworld {
            name
          }
        }
      }
    }
  }

"""

suffix = "Search for the titles of all the stawars films stored in the graphql database that has this schema "

input_message = {
    "role": "user",
    "content": suffix + graphql_fields,
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the titles of all the stawars films stored in the graphql database that has this schema allFilms {
    films {
      title
      director
      releaseDate
      speciesConnection {
        species {
          name
          classification
          homeworld {
            name
          }
        }
      }
    }
  }


================================== Ai Message ==================================
Tool Calls:
  query_graphql (call_tN5A0dBbfOMewuw8Yy13bYpW)
 Call ID: call_tN5A0dBbfOMewuw8Yy13bYpW
  Args:
    tool_input: query { allFilms { films { title } } }
================================= Tool Message =================================
Name: query_graphql

"{\n  \"allFilms\": {\n    \"films\": [\n      {\n        \"title\": \"A New Hope\"\n      },\n      {\n        \"title\": \"The Empire Strikes Back\"\n      },\n      {\n        \"title\": \"Return of the Jedi\"\n      },\n      {\n       